# Biggest winner - biggest loser and other stuff

In [9]:
import pandas as pd
import mysql.connector
pd.options.display.float_format = "{:,.0f}".format

## Connect

In [2]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load Data

In [20]:
df_transaction = pd.read_sql("""SELECT 
      tra_id as id,
      pha_id as pharma_id,
      pha_name as pharma_name,
      rec_id as recipient_id,
      rec_name as recipient_name,
      rec_address as recipient_address,
      rec_location as recipient_location,
      rec_type as recipient_type,
      trc_id as transaction_category_id,
      trc_name as transaction_category_name,
      tra_year as year,
      tra_value as value
  FROM transaction
  LEFT JOIN pharma ON tra_fk_pharma = pha_id 
  LEFT JOIN recipient ON tra_fk_recipient = rec_id
  LEFT JOIN transaction_category ON tra_fk_transaction_category = trc_id;""", con=db_connection)

## Analyze

In [21]:
df_rec = df_transaction.groupby(['recipient_id', 'year']).agg({
    'recipient_name': 'first',
    'recipient_address': 'first',
    'recipient_location': 'first',
    'recipient_type': 'first',
    'value': 'sum'
})

df_pivot = pd.pivot_table(df_rec, values='value', index=['recipient_id', 'recipient_name', 'recipient_type'], columns='year')
df_pivot = df_pivot.reset_index()

### Delta 2018

In [6]:
df_2018 = df_pivot[df_pivot[2017].notna() & df_pivot[2018].notna()].copy()
df_2018['delta_2018'] = round(df_2018[2018] - df_2018[2017])
df_2018['ratio_2018'] = round(100 / df_2018[2017] * df_2018[2018])

## HCO

### Absolut am meisten zugelegt

In [7]:
df_2018 = df_2018.sort_values('delta_2018', ascending=False)
df_2018[df_2018.recipient_type == 'hco'].head(10)

year,recipient_id,recipient_name,recipient_type,2015,2016,2017,2018,delta_2018,ratio_2018
8995,9021,ESMO,hco,3062325.11,10322180.21,10340144.90,12308069.48,1967925.0,119.0
8988,9014,ERS European Respiratory Society ERS Headquarters,hco,2372427.34,3286866.48,2809678.77,4671194.24,1861515.0,166.0
8954,8979,EASL European Association for the Study of the...,hco,1497026.70,2868655.75,2166821.77,3672046.91,1505225.0,169.0
9760,9799,Universität Zürich,hco,812904.59,694331.70,565715.28,1505651.86,939937.0,266.0
9055,9082,Forum für medizinische Fortbildung - FomF GmbH,hco,314224.00,518402.00,761400.80,1566502.16,805101.0,206.0
9007,9033,EULAR - European League Against Rheumatism,hco,987598.00,3383589.20,3541336.97,4301806.18,760469.0,121.0
9004,9030,ETH Zürich,hco,798353.64,626080.00,249716.00,983580.00,733864.0,394.0
8962,8988,ECTRIMS European Committee for Treatment and R...,hco,434606.43,968478.05,666265.95,1190873.15,524607.0,179.0
9095,9122,FOREUM Foundation for Research in Rheumatology,hco,1940172.45,329320.55,307972.44,755983.48,448011.0,245.0
10454,10517,WSO World Stroke Organization,hco,213524.48,98134.00,50007.00,467951.40,417944.0,936.0


### Prozentual am meisten zugelegt

In [27]:
df_2018 = df_2018.sort_values('ratio_2018', ascending=False)
df_2018[df_2018.recipient_type == 'hco'].head(10)

year,recipient_id,recipient_name,recipient_type,2015,2016,2017,2018,delta_2018,ratio_2018
10285,10341,Swiss Oncology & Hematology Congress,hco,NaN,NaN,600.00,358019.00,357419.0,59670.0
9547,9581,Lung Cancer Europe (LUCE),hco,5000.00,28851.00,442.96,209105.28,208662.0,47206.0
9307,9337,High Tech Home Care AG,hco,NaN,NaN,327.04,47300.75,46974.0,14463.0
9290,9320,Hepatitis Schweiz,hco,NaN,NaN,2000.00,230000.00,228000.0,11500.0
10001,10048,Schweizerische MD-PhD Gesellschaft,hco,3586.00,NaN,2000.00,206101.00,204101.0,10305.0
9969,10016,Schweizerische Vereinigung gegen die Osteoporo...,hco,4000.00,36000.00,3000.00,206000.00,203000.0,6867.0
9796,9836,Praxis am Bahnhof,hco,NaN,NaN,200.00,7860.00,7660.0,3930.0
9584,9618,Medical Image Analysis Center (MIAC AG),hco,NaN,22314.84,2497.70,71579.95,69082.0,2866.0
10431,10494,"Viollier AG Kardiologie, Ambulatorium",hco,784.33,NaN,1342.00,35000.00,33658.0,2608.0
10331,10388,"Thurgau Spital, Frauenklinik Spital Frauenfeld",hco,NaN,NaN,2808.00,61325.50,58518.0,2184.0


## HCP

### Absolut am meisten zugelegt

In [28]:
df_2018 = df_2018.sort_values('delta_2018', ascending=False)
df_2018[df_2018.recipient_type == 'hcp'].head(10)

year,recipient_id,recipient_name,recipient_type,2015,2016,2017,2018,delta_2018,ratio_2018
2864,2874,François Mach,hcp,6545.80,11656.83,39239.48,86803.64,47564.0,221.0
6934,6950,Rolf A. Stahel,hcp,81973.84,75887.00,98930.11,141462.94,42533.0,143.0
5428,5443,Matti Aapro,hcp,97214.55,34925.87,39577.09,72624.21,33047.0,184.0
7862,7880,Thomas Szucs,hcp,1000.00,1800.00,1000.00,32317.24,31317.0,3232.0
6733,6749,Reinhard Dummer,hcp,NaN,2000.00,7914.10,38902.88,30989.0,492.0
117,121,Albina Nowak,hcp,7373.51,NaN,600.00,25828.84,25229.0,4305.0
134,138,Alessandra Curioni,hcp,2324.00,642.00,8327.00,31162.95,22836.0,374.0
6857,6873,Robert Steffen,hcp,6000.00,1200.00,16337.50,38066.00,21728.0,233.0
7247,7263,Serge Ferrari,hcp,NaN,3157.00,1500.00,21214.23,19714.0,1414.0
5697,5712,Miklos Pless,hcp,8709.27,17827.00,12395.02,31452.66,19058.0,254.0


### Prozentual am meisten zugelegt

In [29]:
df_2018 = df_2018.sort_values('ratio_2018', ascending=False)
df_2018[df_2018.recipient_type == 'hcp'].head(10)

year,recipient_id,recipient_name,recipient_type,2015,2016,2017,2018,delta_2018,ratio_2018
117,121,Albina Nowak,hcp,7373.51,NaN,600.00,25828.84,25229.0,4305.0
7862,7880,Thomas Szucs,hcp,1000.00,1800.00,1000.00,32317.24,31317.0,3232.0
6310,6326,Peter Bartel,hcp,1450.30,1239.46,83.42,1949.98,1867.0,2338.0
521,527,Angela Pugliesi,hcp,NaN,1500.00,400.20,9167.26,8767.0,2291.0
1975,1984,Jan Izakovic,hcp,7933.37,2471.06,500.00,10432.25,9932.0,2086.0
5662,5677,Michel Zuber,hcp,NaN,NaN,1000.00,20000.00,19000.0,2000.0
2378,2388,Doris Zürcher,hcp,NaN,NaN,500.00,8309.30,7809.0,1662.0
7157,7173,Sandrine Zweifel,hcp,2000.00,1830.98,750.00,12314.05,11564.0,1642.0
1006,1014,Photis Beris,hcp,7914.57,6673.90,160.36,2623.95,2464.0,1636.0
628,634,Anne Rosselet,hcp,9802.67,2472.93,528.67,8523.12,7994.0,1612.0


## Absolut tops

In [12]:
df_2018 = df_2018.sort_values(2018, ascending=False)
df_2018[df_2018.recipient_type == 'hco'].head(10)

year,recipient_id,recipient_name,recipient_type,2015,2016,2017,2018,delta_2018,ratio_2018
8995,9021,ESMO,hco,"3,062,325","10,322,180","10,340,145","12,308,069","1,967,925",119
8988,9014,ERS European Respiratory Society ERS Headquarters,hco,"2,372,427","3,286,866","2,809,679","4,671,194","1,861,515",166
9007,9033,EULAR - European League Against Rheumatism,hco,"987,598","3,383,589","3,541,337","4,301,806","760,469",121
8954,8979,EASL European Association for the Study of the...,hco,"1,497,027","2,868,656","2,166,822","3,672,047","1,505,225",169
9028,9054,EXCEMED - Excellence in Medical Education,hco,"4,043,310","3,137,022","4,694,351","2,722,032","-1,972,319",58
8916,8938,Insel Gruppe AG,hco,"1,994,105","2,419,922","2,709,783","2,562,998","-146,785",95
8693,8713,Universitätsspital Basel,hco,"1,695,838","2,010,889","2,607,496","2,511,371","-96,125",96
8952,8977,EADV European Academy of Dermatology and Vener...,hco,"1,135,978","837,189","1,900,173","2,254,468","354,296",119
8800,8822,Centre Hospitalier Universitaire Vaudois CHUV,hco,"1,685,507","2,025,378","2,462,314","2,167,948","-294,366",88
8914,8936,UniversitätsSpital Zürich,hco,"1,321,398","1,441,830","2,173,823","2,047,589","-126,234",94


## Franken pro Empfänger pro Pharma

In [39]:
df_hcp = df_transaction[df_transaction.recipient_type == 'hcp']
df_hcp = df_hcp.groupby(['pharma_name', 'year']).agg({'value': 'sum', 'recipient_id': 'count'})
df_pivot = pd.pivot_table(df_hcp, values=['value', 'recipient_id'], columns='year', index='pharma_name')
df_pivot['2015_avg'] = df_pivot['value'][2015] / df_pivot['recipient_id'][2015]
df_pivot['2016_avg'] = df_pivot['value'][2016] / df_pivot['recipient_id'][2016]
df_pivot['2017_avg'] = df_pivot['value'][2017] / df_pivot['recipient_id'][2017]
df_pivot['2018_avg'] = df_pivot['value'][2018] / df_pivot['recipient_id'][2018]

df_pivot = df_pivot.sort_values(['2018_avg'], ascending=False)

df_pivot.head(10)

recipient_id                  value                  \
year                              2015 2016 2017 2018    2015    2016    2017   
pharma_name                                                                     
Neurim Pharmaceuticals AG            1  nan    1    1     800     nan     900   
Takeda Pharmaceutical              416  171   80   51 329,287 179,202 118,882   
Sandoz Pharmaceuticals AG          205  191   28   31 129,196 112,734  57,976   
MSD                                105  145  137  265 240,423 271,219 257,042   
Mepha                              146  303   70   55 145,223 173,109  92,663   
Stallergenes Greer                 104  104   38   17  60,726  60,726  22,466   
Sanofi                             182  151  215  159 191,170 162,840 218,950   
Shire Pharmaceuticals              nan    6   30   18     nan  10,584  32,794   
Galderma                            48   57   56   64  59,463  95,745  80,086   
Hoffmann-La Roche                  493  429  448  413 457,103 465,683 448,393   

                                  2015_avg 2016_avg 2017_avg 2018_avg  
year                         2018                                      
pharma_name                                                            
Neurim Pharmaceuticals AG   2,500      800      nan      900    2,500  
Takeda Pharmaceutical     124,074      792    1,048    1,486    2,433  
Sandoz Pharmaceuticals AG  63,795      630      590    2,071    2,058  
MSD                       480,680    2,290    1,870    1,876    1,814  
Mepha                      91,316      995      571    1,324    1,660  
Stallergenes Greer         24,073      584      584      591    1,416  
Sanofi                    224,902    1,050    1,078    1,018    1,414  
Shire Pharmaceuticals      25,347      nan    1,764    1,093    1,408  
Galderma                   81,302    1,239    1,680    1,430    1,270  
Hoffmann-La Roche         524,206      927    1,086    1,001    1,269

In [32]:
df_pivot['recipient_id'][2015]

pharma_name
A. Menarini AG                     386
AbbVie                             511
Actelion                           178
Alcon                               15
Allergan                            41
Almirall                            49
Amgen                              404
Astellas Pharma AG                 164
AstraZeneca                        201
Basilea Pharmaceutica              nan
Baxalta                              4
Baxter AG                           75
Bayer AG                           426
Biogen                              53
Biotest AG                           9
Boehringer Ingelheim               171
Bristol-Myers Squibb SA            110
CSL Behring                         52
Celgene                            350
Daiichi Sankyo                      82
Eisai                               15
Eli Lilly SA                       189
Ferring Pharmaceuticals             80
Future Health Pharma GmbH           35
Galderma                            48
Gilead Scienc

In [16]:
df_hcp.columns

Index(['id', 'pharma_id', 'pharma_name', 'recipient_id', 'recipient_name',
       'recipient_address', 'recipient_location', 'recipient_type',
       'transaction_category_id', 'transaction_category_name', 'year',
       'value'],
      dtype='object')

## Ärzte Honorare
**ToDo: Zählen, von wievielen Pharmafirmen bekommen. Suchen, wer am meisten nur von einer Firma bekommen hat**

In [50]:
df_hcp = df_transaction[df_transaction.recipient_type == 'hcp']
df_hcp = df_hcp[df_hcp.transaction_category_name == 'fees']
df_grouped = df_hcp.groupby(['recipient_id']).agg({'recipient_name': 'first', 'value': 'sum'})
df_grouped = df_grouped.sort_values('value', ascending=False)
df_grouped.head(10)


,recipient_name,value
recipient_id,,
6950,Rolf A. Stahel,"267,365"
5443,Matti Aapro,"174,374"
1987,Jan Steffel,"143,220"
1156,Wolf-Henning Boehncke,"136,012"
1140,Johannes Bitzer,"132,897"
3017,Gerhard Rogler,"126,471"
2205,Jean Dudler,"110,825"
2370,Marc Donat,"99,569"
2874,François Mach,"96,121"


In [44]:
df_hcp.transaction_category_name.unique()

array(['registration_fees', 'travel_accommodation', 'fees',
       'related_expenses', 'sponsorship', 'donations_grants'],
      dtype=object)

In [46]:
df_hcp[df_hcp.recipient_name.str.contains('Savaskan')]

,id,pharma_id,pharma_name,recipient_id,recipient_name,recipient_address,recipient_location,recipient_type,transaction_category_id,transaction_category_name,year,value
7971,7972,15,Boehringer Ingelheim,2449,Egemen Savaskan,Minervastrasse 145,Zürich,hcp,5,fees,2017,"1,050"
7972,7973,57,Vifor Pharma,2449,Egemen Savaskan,Minervastrasse 145,Zürich,hcp,5,fees,2016,"1,000"
7973,7974,57,Vifor Pharma,2449,Egemen Savaskan,Minervastrasse 145,Zürich,hcp,5,fees,2015,"1,000"
24510,24511,21,Eli Lilly SA,2449,Egemen Savaskan,Minervastrasse 145,Zürich,hcp,5,fees,2016,"2,600"
